In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json


def get_filepath(state):
    return '../data/{state}/'.format(state=state)
def get_filepath_city(city):
    return '../data/' + city + '-bike/'

CITIES = ['dc', 'boston', 'chicago', 'nyc', 'philadelphia']
STATES = ['dc', 'ma', 'il', 'ny', 'pa']
START_YEARS = [2010, 2011, 2013, 2013, 2016]
YEARS_LISTS = {'dc': range(2010, 2023),
        'boston': range(2011, 2023),
        'chicago': range(2013, 2023),
        'nyc': range(2013, 2023),
        'philadelphia': range(2016, 2023)
}
                    
                    

all_area =  True



In [ ]:

def calc_gini(vals):
    
        length  = len(vals)
        aggregate_density = (sum(vals))
        y = []
        for i, val in enumerate(vals):
            amount = sum(vals[:i+1])/aggregate_density
            y.append(amount*100)
            if i == int(length/2) and year == 2022:
                print(1-amount, CITIES[idx])
            
                
            
        cutoffs[CITIES[idx]].append(int((length*2)/4))
        
        # print(length)
        x = [i/length*100 for i in range(length)]

        lorence_a = np.trapz(y, x)
        total_a = np.trapz([0, 100], [0, 100])
        gini = (total_a - lorence_a)/total_a
        return x, y, gini

fig, axs = plt.subplots(nrows = 1, ncols = 5, figsize = (10, 2.4), sharey = True, dpi = 720)
ginis = {x: [] for x in CITIES}
tracts = {x: [] for x in CITIES}
cutoffs = {x: [] for x in CITIES}
avg_incomes = {x: [] for x in CITIES}
avg_race = {x:[] for x in CITIES}
for idx, STATE in enumerate(STATES):
    data_file = pd.read_csv(get_filepath(STATE) + f'{STATE}_preprocessed_data.csv')
    y_count = 0
    axs[idx].plot([0, 100], [0, 100], linestyle = 'dashed', color = 'black')
    YEARS = range(START_YEARS[idx], 2023)

    for year in YEARS:
        temp_data = data_file.loc[data_file['year'] == year]

        data_list = list(temp_data.to_records())
        data_list.sort(key = lambda x: x[4]/x[5])
        households = [x[5] for x in data_list]
        num_white = [x[7] for x in data_list]
        percent_white  = sum(num_white)/sum(households)
        income = [x[6] for x in data_list]
        avg_incomes[CITIES[idx]].append(sum(income)/len(income))
        avg_race[CITIES[idx]].append(percent_white)

        if not all_area:
            data_list = [x for x in data_list if x[18]]
            tracts[CITIES[idx]].append(data_list)

       
        vals = [x[4]/x[5] for x in data_list]
        x, y, gini = calc_gini(vals)
        
        axs[idx].plot(x, y, alpha = 1 - (2023 - year)/(1.1*len(YEARS)), color = 'b')
        
        axs[idx].set_xlim(0, 100)
        axs[idx].set_ylim(0, 100)
        ginis[CITIES[idx]].append(gini)

        y_count += 1
    axs[idx].set_title(CITIES[idx].upper(), fontsize = 12)
    axs[idx].set_aspect('equal')
    axs[idx].set_xticks([0, 25, 50, 75, 100])
    axs[idx].set_xticklabels([0, 25, 50, 75, 100], rotation=90)

# print(cutoffs)
# print(avg_incomes)
# print(avg_race)
# print(ginis)
fig.show()

fig.supylabel('% Accessibility', fontsize = 12)
axs[2].set_xlabel('% Census Tracts', fontsize = 12, labelpad=7)
# fig.tight_layout()
fig.subplots_adjust(left =.08)
fig.subplots_adjust(bottom = .25)
fig.subplots_adjust(wspace=.01, hspace=0)


    

In [ ]:

fig, axs = plt.subplots(nrows = 2, ncols = 5, figsize = (10, 5),sharey = 'row',sharex = 'col',  dpi = 600)
plt.style.use('default')
count = 0
for CITY in cutoffs:
    x = YEARS_LISTS[CITY]
    y1 = []
    y2 = []
    y3 = []
    y4 = []
    for idx, cutoff in enumerate(cutoffs[CITY]):
        service_area_households = [x[5] for x in tracts[CITY][idx]]
        service_area_num_white = [x[7] for x in tracts[CITY][idx]]
        service_area_percent_white = sum(service_area_num_white)/sum(service_area_households)
        avg_pct_white_top = sum(service_area_num_white[cutoff:])/sum(service_area_households[cutoff:])
        
        income = [x[6] for x in tracts[CITY][idx]]
        avg_income = avg_incomes[CITY][idx]
        
        avg_income_top = sum(income[cutoff:])/len(income[cutoff:])
        y1.append(service_area_percent_white*100)
        y2.append((avg_pct_white_top)*100)
        y3.append(sum(income)/len(income))
        y4.append(avg_income_top)
    
    axs[1, count].plot(x, y2, color = 'purple', label = 'Top 50% in service area')
    axs[0, count].plot(x, [val/1000 for val in y4], color = 'green', label = 'Top 50% in service area')
    axs[1, count].plot(x, y1, color = 'grey', linestyle = 'dashed', label = 'Service area')
    axs[0, count].plot(x, [val/1000 for val in y3], color = 'grey', linestyle = 'dashed', label = 'Avg in service area')
    axs[1, count].plot(x, [val*100 for val in avg_race[CITY]], linestyle = 'dashed', color = 'black', label = 'Study area')
 
    axs[0, count].plot(x, [val/1000 for val in avg_incomes[CITY]], linestyle = 'dashed', color = 'black', label = 'Avg in study area')
    

    axs[1, count].set_ylim(0, 85)
    axs[1, count].set_xlim(2010, 2022)
    axs[0, count].set_xlim(2010, 2022)
    axs[0, count].set_ylim(0, 120)
    axs[0, count].set_title(CITY.upper(), fontsize = 13)
    axs[1, count].set_xticks([2012, 2014, 2016, 2018, 2020, 2022])
    axs[1, count].set_xticklabels([2012, 2014, 2016, 2018, 2020, 2022], rotation=90)
    
    

    count += 1
axs[1, 0].set_xticks([2010, 2012, 2014, 2016, 2018, 2020, 2022])
axs[1, 0].set_xticklabels([2010, 2012, 2014, 2016, 2018, 2020, 2022], rotation=90)
axs[1, 0].set_ylabel('% White', fontsize = 'large')
axs[0, 0].set_ylabel('Income in Thousands ($)', fontsize = 'large')
fig.supxlabel('Year', fontsize = 'x-large')
fig.tight_layout()
fig.subplots_adjust(left=.08)
fig.subplots_adjust(right = .93)
fig.subplots_adjust(wspace=.08, hspace=.08)


# fig2.suptitle("Difference Between the Average Income of the Census Tracts Containing the Top 25% of Accessibility and the Avergage Income of Each City's Service Area Over Time", fontsize = 'xx-large')


In [ ]:
import unittest

class TestGini(unittest.TestCase):
    
    def test_gini_calculation_all_equal(self):
        vals = [1 for _ in range(5000)]
        self.assertAlmostEqual(calc_gini(vals)[2], 0, delta = .001)
    
    def test_gini_calculation_very_unequal(self):
        vals = [0 for _ in range(5000)]
        vals.append(1)
        self.assertAlmostEqual(calc_gini(vals)[2], 1, delta = .001)

    def test_gini_calculation_quadratic(self):
        vals = [x**2 for x in range(1000)]
        self.assertAlmostEqual(calc_gini(vals)[2], .5, delta = .001)
    

        
res = unittest.main(argv=[''], verbosity=3, exit=False)

# if we want our notebook to stop processing due to failures, we need a cell itself to fail
assert len(res.result.failures) == 0
